In [1]:
import cv2

In [1]:
def add_breach(im, x, y, size):
    assert x-size/2 >= 0 and x+size/2 <= im[0].size and y-size/2 >= 0 and y+size/2 <= im[1].size
    raze = cv2.imread("Characters/Breach.png")[:, :, ::-1]
    raze = cv2.resize(raze, (size, size))
    im[x-size//2 : x+size//2, y-size//2 : y+size//2, :] = raze
    return im

def add_brimstone(im, x, y, size):
    assert x-size/2 >= 0 and x+size/2 <= im[0].size and y-size/2 >= 0 and y+size/2 <= im[1].size
    raze = cv2.imread("Characters/Brimstone.png")[:, :, ::-1]
    raze = cv2.resize(raze, (size, size))
    im[x-size//2 : x+size//2, y-size//2 : y+size//2, :] = raze
    return im

def add_cypher(im, x, y, size):
    assert x-size/2 >= 0 and x+size/2 <= im[0].size and y-size/2 >= 0 and y+size/2 <= im[1].size
    raze = cv2.imread("Characters/Cypher.png")[:, :, ::-1]
    raze = cv2.resize(raze, (size, size))
    im[x-size//2 : x+size//2, y-size//2 : y+size//2, :] = raze
    return im

def add_jett(im, x, y, size):
    assert x-size/2 >= 0 and x+size/2 <= im[0].size and y-size/2 >= 0 and y+size/2 <= im[1].size
    raze = cv2.imread("Characters/Jett.png")[:, :, ::-1]
    raze = cv2.resize(raze, (size, size))
    im[x-size//2 : x+size//2, y-size//2 : y+size//2, :] = raze
    return im

def add_omen(im, x, y, size):
    assert x-size/2 >= 0 and x+size/2 <= im[0].size and y-size/2 >= 0 and y+size/2 <= im[1].size
    raze = cv2.imread("Characters/Omen.png")[:, :, ::-1]
    raze = cv2.resize(raze, (size, size))
    im[x-size//2 : x+size//2, y-size//2 : y+size//2, :] = raze
    return im

def add_phoenix(im, x, y, size):
    assert x-size/2 >= 0 and x+size/2 <= im[0].size and y-size/2 >= 0 and y+size/2 <= im[1].size
    raze = cv2.imread("Characters/Phoenix.png")[:, :, ::-1]
    raze = cv2.resize(raze, (size, size))
    im[x-size//2 : x+size//2, y-size//2 : y+size//2, :] = raze
    return im

def add_raze(im, x, y, size):
    assert x-size/2 >= 0 and x+size/2 <= im[0].size and y-size/2 >= 0 and y+size/2 <= im[1].size
    raze = cv2.imread("Characters/Raze.png")[:, :, ::-1]
    raze = cv2.resize(raze, (size, size))
    im[x-size//2 : x+size//2, y-size//2 : y+size//2, :] = raze
    return im

def add_reyna(im, x, y, size):
    assert x-size/2 >= 0 and x+size/2 <= im[0].size and y-size/2 >= 0 and y+size/2 <= im[1].size
    raze = cv2.imread("Characters/Reyna.png")[:, :, ::-1]
    raze = cv2.resize(raze, (size, size))
    im[x-size//2 : x+size//2, y-size//2 : y+size//2, :] = raze
    return im

def add_sage(im, x, y, size):
    assert x-size/2 >= 0 and x+size/2 <= im[0].size and y-size/2 >= 0 and y+size/2 <= im[1].size
    raze = cv2.imread("Characters/Sage.png")[:, :, ::-1]
    raze = cv2.resize(raze, (size, size))
    im[x-size//2 : x+size//2, y-size//2 : y+size//2, :] = raze
    return im

def add_sova(im, x, y, size):
    assert x-size/2 >= 0 and x+size/2 <= im[0].size and y-size/2 >= 0 and y+size/2 <= im[1].size
    raze = cv2.imread("Characters/Sova.png")[:, :, ::-1]
    raze = cv2.resize(raze, (size, size))
    im[x-size//2 : x+size//2, y-size//2 : y+size//2, :] = raze
    return im

def add_viper(im, x, y, size):
    assert x-size/2 >= 0 and x+size/2 <= im[0].size and y-size/2 >= 0 and y+size/2 <= im[1].size
    raze = cv2.imread("Characters/Viper.png")[:, :, ::-1]
    raze = cv2.resize(raze, (size, size))
    im[x-size//2 : x+size//2, y-size//2 : y+size//2, :] = raze
    return im

In [3]:
adds = [add_breach, add_brimstone, add_cypher, add_jett, add_omen, add_phoenix, add_raze, add_reyna, add_sage, add_sova, add_viper]